###**Prediction of English Language Proficiency Assessments for California (ELPAC) Scores Using Machine Learning Approach**
###**DATA ACQUISTION**

###**TEAM MEMBERS: TEAM 3: EMMA OO, LUKE AWINO, OSCAR GIL**
###**Applied Data Science Master’s Program, Shiley Marcos School of Engineering / University of San Diego**
###**DATE: 11/12/2022**

In [ ]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import numpy as np
import datetime
import sqlite3

In [ ]:
ELPAC_file_path = '/Users/oscargil/Desktop/ADS-599B/Raw_Files/ELPAC.xlsx'

#### Import files from Excel file containing ELPAC results

In [ ]:
# Import ELPAC data into dataframes
elpac1718 = pd.read_excel(open(ELPAC_file_path, 'rb'), sheet_name='2017-2018', dtype=str)


# Drop identified columns
elpac1718.drop(elpac1718.columns[[0, 1, 5, 7, 8, 9, 10]], axis=1, inplace=True)

In [ ]:
# Import ELPAC data into dataframes
elpac1819 = pd.read_excel(open(ELPAC_file_path, 'rb'), sheet_name='2018-2019', dtype=str)


# Drop identified columns
elpac1819.drop(elpac1819.columns[[0, 1, 5, 7, 8, 9, 10]], axis=1, inplace=True)

In [ ]:
# Import ELPAC data into dataframes
elpac1920 = pd.read_excel(open(ELPAC_file_path, 'rb'), sheet_name='2019-2020', dtype=str)


# Drop identified columns
elpac1920.drop(elpac1920.columns[[0, 1, 5, 7, 8, 9, 10]], axis=1, inplace=True)

In [ ]:
# Import ELPAC data into dataframes
elpac2021 = pd.read_excel(open(ELPAC_file_path, 'rb'), sheet_name='2020-2021', dtype=str)


# Drop identified columns
elpac2021.drop(elpac2021.columns[[0, 1, 6, 8, 9, 10, 11]], axis=1, inplace=True)

In [ ]:
# Import ELPAC data into dataframes
elpac2122 = pd.read_excel(open(ELPAC_file_path, 'rb'), sheet_name='2021-2022', dtype=str)

# Remove 44 results associated to the Alternative ELPAC
elpac2122 = elpac2122.query("RecordType == '21'")


# Drop identified columns
elpac2122.drop(elpac2122.columns[[1, 5, 7, 8, 9, 17, 19, 21, 54, 55, 56, 57, 58, 61, 62, 63, 64, 70, 71, 72, 73, 79, 80, 81, 82, 88, 89, 90, 91, 97, 98, 99, 100]], axis=1, inplace=True)

#### Create a unique list of all SSIDs and assign them the Pandas Dataframe index number as their de-identified number

In [ ]:
# Create data frame of SSIDs to de-identify
ssids = pd.DataFrame(pd.concat([elpac1718.elpac_2018_ssid, elpac1819.elpac_2019_ssid, elpac1920.elpac_2020_ssid, elpac2021.elpac_2021_ssid, elpac2122.SSID], axis=0))

# Unique SSID values
ssid_deidentify = pd.DataFrame(ssids[0].unique())

ssid_deidentify.columns = ['SSID']

# Assign Data frame index as de-identified ID
ssid_deidentify['Stu_deID'] =  ssid_deidentify.index

In [ ]:
# Create local file to have as reference to Stu_deID
ssid_deidentify.to_csv('/Users/oscargil/Desktop/ADS-599B/Raw_Files/ssid_deidentify.csv', encoding='utf-8', index=False)

#### Merge de-identified ID into ELPAC dataframes, then dropped the SSID field containing the real values

In [ ]:
# Merge in de-identier
elpac1718 = elpac1718.merge(ssid_deidentify, how="inner", left_on='elpac_2018_ssid', right_on='SSID')

# Drop SSID columns
elpac1718.drop(elpac1718.columns[[3, 57]], axis=1, inplace=True)

In [ ]:
# Merge in de-identier
elpac1819 = elpac1819.merge(ssid_deidentify, how="inner", left_on='elpac_2019_ssid', right_on='SSID')

# Drop SSID columns
elpac1819.drop(elpac1819.columns[[3, 54]], axis=1, inplace=True)

In [ ]:
# Merge in de-identier
elpac1920 = elpac1920.merge(ssid_deidentify, how="inner", left_on='elpac_2020_ssid', right_on='SSID')

# Drop SSID columns
elpac1920.drop(elpac1920.columns[[3, 79]], axis=1, inplace=True)

In [ ]:
# Merge in de-identier
elpac2021 = elpac2021.merge(ssid_deidentify, how="inner", left_on='elpac_2021_ssid', right_on='SSID')

# Drop SSID columns
elpac2021.drop(elpac2021.columns[[4, 120]], axis=1, inplace=True)

In [ ]:
# Merge in de-identier
elpac2122 = elpac2122.merge(ssid_deidentify, how="inner", left_on='SSID', right_on='SSID')

# Drop SSID columns
elpac2122.drop(elpac2122.columns[[3]], axis=1, inplace=True)

### Add AcademicYear column to each ELPAC file, to join into attendance files

In [ ]:
# Add AcademicYear column
elpac1718['AcademicYear'] = '2017-2018'
elpac1819['AcademicYear'] = '2018-2019'
elpac1920['AcademicYear'] = '2019-2020'
elpac2021['AcademicYear'] = '2020-2021'
elpac2122['AcademicYear'] = '2021-2022'

### Identify features to keep from ELPAC files

In [ ]:
# Subset ELPAC files
elpac1718 = elpac1718.iloc[:,[56, 57, 3, 4, 7, 10, 11]]
elpac1819 = elpac1819.iloc[:,[53, 54, 3, 4, 7, 10, 11]]
elpac1920 = elpac1920.iloc[:,[78, 79, 3, 6, 7, 8, 9]]
elpac2021 = elpac2021.iloc[:,[119, 120, 4, 5, 8, 9, 10]]
elpac2122 = elpac2122[['Stu_deID', 'AcademicYear','DateofBirth', 'CALPADSGrade', 'FinalTestCompletedDate', 'OverallScaleScore', 'OverallPL']]

# Rename columns
elpac_columns = ['Stu_deID', 'AcademicYear', 'DOB', 'GradeLevel', 'TestDate', 'OverallScore', 'OverallLevel']

elpac1718.columns = elpac_columns
elpac1819.columns = elpac_columns
elpac1920.columns = elpac_columns
elpac2021.columns = elpac_columns
elpac2122.columns = elpac_columns

In [ ]:
# Data conversions - DOB to datetime
elpac1718['DOB'] = pd.to_datetime(elpac1718['DOB'])
elpac1819['DOB'] = pd.to_datetime(elpac1819['DOB'])
elpac1920['DOB'] = pd.to_datetime(elpac1920['DOB'])
elpac2021['DOB'] = pd.to_datetime(elpac2021['DOB'])
elpac2122['DOB'] = pd.to_datetime(elpac2122['DOB'])

In [ ]:
# Data conversions - TestDate to datetime
elpac1718['TestDate'] = pd.to_datetime(elpac1718['TestDate'])
elpac1819['TestDate'] = pd.to_datetime(elpac1819['TestDate'])
elpac1920['TestDate'] = pd.to_datetime(elpac1920['TestDate'])
elpac2021['TestDate'] = pd.to_datetime(elpac2021['TestDate'])
elpac2122['TestDate'] = pd.to_datetime(elpac2122['TestDate'])

In [ ]:
# Data conversions - GradeLevel clean up, eliminate extra zero, convert KN to 0
elpac1718['GradeLevel'] = elpac1718.GradeLevel.replace('KN', '0').astype(int)
elpac1819['GradeLevel'] = elpac1819.GradeLevel.replace('KN', '0').astype(int)
elpac1920['GradeLevel'] = elpac1920.GradeLevel.replace('KN', '0').astype(int)
elpac2021['GradeLevel'] = elpac2021.GradeLevel.replace('KN', '0').astype(int)
elpac2122['GradeLevel'] = elpac2122.GradeLevel.replace('KN', '0').astype(int)

In [ ]:
elpac1718['OverallScore'] = elpac1718.OverallScore.replace('NS',0).fillna(0).astype(int)
elpac1819['OverallScore'] = elpac1819.OverallScore.replace('NS',0).fillna(0).astype(int)
elpac1920['OverallScore'] = elpac1920.OverallScore.replace('NS',0).fillna(0).astype(int)
elpac2021['OverallScore'] = elpac2021.OverallScore.replace('NS',0).fillna(0).astype(int)
elpac2122['OverallScore'] = elpac2122.OverallScore.replace('NS',0).fillna(0).astype(int)

elpac1718['OverallLevel'] = elpac1718.OverallLevel.replace('NS',0).fillna(0).astype(int)
elpac1819['OverallLevel'] = elpac1819.OverallLevel.replace('NS',0).fillna(0).astype(int)
elpac1920['OverallLevel'] = elpac1920.OverallLevel.replace('NS',0).fillna(0).astype(int)
elpac2021['OverallLevel'] = elpac2021.OverallLevel.replace('NS',0).fillna(0).astype(int)
elpac2122['OverallLevel'] = elpac2122.OverallLevel.replace('NS',0).fillna(0).astype(int)

In [ ]:
# Update Elpac file containing all years
elpac = pd.concat([elpac1718, elpac1819, elpac1920, elpac2021, elpac2122], axis=0)

#### Feature Engineering Ideas

* Day of the week
* Grade Level + percentage of days enrolled (so for example, 4th grade would not be represented by a '4', instead, by '4.80' - a 4th grader who attended 80% of the time)
* Number of Levels a student progressed from previous ELPAC (17-18 a student who scores a Level 1, then in 18-19 scoed a Level 3; the student progressed 2 levels)
* Age of Student at time of test
* Teacher information (years teaching)
* Total in Household
* Household income
* Homeless

In [ ]:
# Feature Engineering

# TestDayName
elpac['TestDayName'] = elpac.TestDate.dt.day_name()

In [ ]:
# Age at time of test
elpac['TestAge'] = (elpac.TestDate - elpac.DOB) / np.timedelta64(1, 'Y')

In [ ]:
elpac

,Stu_deID,AcademicYear,DOB,GradeLevel,TestDate,OverallScore,OverallLevel,TestDayName,TestAge
0,0,2017-2018,2011-07-04,1,2018-05-02,1503,4,Wednesday,6.828340
1,1,2017-2018,2008-07-28,4,2018-03-16,1535,3,Friday,9.631957
2,2,2017-2018,2011-08-13,1,2018-02-12,1449,3,Monday,6.502529
3,3,2017-2018,2008-05-06,4,2018-04-02,1480,2,Monday,9.905748
4,4,2017-2018,2012-04-13,0,2018-05-23,1461,4,Wednesday,6.108271
...,...,...,...,...,...,...,...,...,...
2081,4785,2021-2022,2016-10-24,0,2022-05-26,1344,1,Thursday,5.585330
2082,3003,2021-2022,2013-10-12,2,2022-05-23,1481,3,Monday,8.610718
2083,4786,2021-2022,2010-06-18,6,2022-05-31,1412,1,Tuesday,11.950964
2084,4787,2021-2022,2016-10-26,0,2022-05-17,1381,2,Tuesday,5.555213


### Attendance Files

In [ ]:
Attendance_file_path = '/Users/oscargil/Desktop/ADS-599B/Raw_Files/'

In [ ]:
# Load attendance data
att_1718 = pd.read_csv(Attendance_file_path + 'STAS_ODS_1718.txt', sep='^', dtype=str, header=None)
att_1819 = pd.read_csv(Attendance_file_path + 'STAS_ODS_1819.txt', sep='^', dtype=str, header=None)
att_1920 = pd.read_csv(Attendance_file_path + 'STAS_ODS_1920.txt', sep='^', dtype=str, header=None)
att_2021 = pd.read_csv(Attendance_file_path + 'STAS_ODS_2021.txt', sep='^', dtype=str, header=None)
att_2122 = pd.read_csv(Attendance_file_path + 'STAS_ODS_2122.txt', sep='^', dtype=str, header=None)

In [ ]:
# columns to keep
att_1718 = att_1718.iloc[:,[5, 6, 14, 15]].fillna(0.00)
att_1819 = att_1819.iloc[:,[5, 6, 14, 15]].fillna(0.00)
att_1920 = att_1920.iloc[:,[5, 6, 14, 15]].fillna(0.00)
att_2021 = att_2021.iloc[:,[5, 6, 14, 15]].fillna(0.00)
att_2122 = att_2122.iloc[:,[5, 6, 14, 15]].fillna(0.00)

In [ ]:
# Assign column names
att_columns = ['AcademicYear', 'SSID', 'ExpectedAttendanceDays', 'DaysAttended']

att_1718.columns = att_columns
att_1819.columns = att_columns
att_1920.columns = att_columns
att_2021.columns = att_columns
att_2122.columns = att_columns

In [ ]:
# Data conversions
att_1718['ExpectedAttendanceDays'] = att_1718.ExpectedAttendanceDays.astype(float)
att_1819['ExpectedAttendanceDays'] = att_1819.ExpectedAttendanceDays.astype(float)
att_1920['ExpectedAttendanceDays'] = att_1920.ExpectedAttendanceDays.astype(float)
att_2021['ExpectedAttendanceDays'] = att_2021.ExpectedAttendanceDays.astype(float)
att_2122['ExpectedAttendanceDays'] = att_2122.ExpectedAttendanceDays.astype(float)

In [ ]:
# Data conversions
att_1718['DaysAttended'] = att_1718.DaysAttended.astype(float)
att_1819['DaysAttended'] = att_1819.DaysAttended.astype(float)
att_1920['DaysAttended'] = att_1920.DaysAttended.astype(float)
att_2021['DaysAttended'] = att_2021.DaysAttended.astype(float)
att_2122['DaysAttended'] = att_2122.DaysAttended.astype(float)

In [ ]:
# Att into one file
att = pd.concat([att_1718, att_1819, att_1920, att_2021, att_2122], axis =0)

### Student demographics and Teacher data

In [ ]:
# Informaation from SQL query in Data Server with file name "CAPSTONE SQL code.sql"
teacher_stuDemographics = pd.read_csv('/Users/oscargil/Desktop/ADS-599B/Raw_Files/teacher_stuDemographics.csv', dtype=str)

In [ ]:
# Unique Schoolname values - only present in the 2021-2022 file
schools_deidentify = pd.DataFrame(teacher_stuDemographics.SchoolName.unique())

schools_deidentify.columns = ['SchoolName']

# Assign Data frame index as de-identified ID
schools_deidentify['School_deID'] =  schools_deidentify.index

# Merge in de-identier
teacher_stuDemographics = teacher_stuDemographics.merge(schools_deidentify, how="inner", left_on='SchoolName', right_on='SchoolName')

# Drop school name column
teacher_stuDemographics.drop(teacher_stuDemographics.columns[[2]], axis=1, inplace=True)

# Create local file to have as reference to School_deID
schools_deidentify.to_csv('/Users/oscargil/Desktop/ADS-599B/Raw_Files/schools_deidentify.csv', encoding='utf-8', index=False)


In [ ]:
# Incorporate deidentified student ID
teacher_stuDemographics = teacher_stuDemographics.merge(ssid_deidentify, how="inner", left_on='SSID', right_on='SSID')

# Drop student ID column
teacher_stuDemographics.drop(teacher_stuDemographics.columns[[1]], axis=1, inplace=True)

In [ ]:
# Update elpac dataframe
elpac = elpac.merge(teacher_stuDemographics, how="inner", left_on=('AcademicYear', 'Stu_deID'), right_on=('AcademicYear', 'Stu_deID'))

### SQLite

In [ ]:
cnn = sqlite3.connect('/Users/oscargil/Desktop/ADS-599B/Raw_Files/db.db')

In [ ]:
# Load dataframes into sql
att.to_sql("att", cnn, if_exists='replace')
ssid_deidentify.to_sql("ssid", cnn, if_exists='replace')

In [ ]:
%load_ext sql

In [ ]:
%sql sqlite:////Users/oscargil/Desktop/ADS-599B/Raw_Files/db.db

'Connected: @/Users/oscargil/Desktop/ADS-599B/Raw_Files/db.db'

In [ ]:
%%sql

select s.Stu_deID, AcademicYear, sum(ExpectedAttendanceDays) as ExpectedAttendanceDays, sum(DaysAttended) as DaysAttended
, sum(DaysAttended) / sum(ExpectedAttendanceDays) as AttendedPct
, sum(ExpectedAttendanceDays) / 180.0 as EnrolledPct
from att as a 
join ssid as s 
    on a.SSID = s.SSID
group by a.SSID, a.AcademicYear
limit 10

 * sqlite:////Users/oscargil/Desktop/ADS-599B/Raw_Files/db.db
Done.


Stu_deID,AcademicYear,ExpectedAttendanceDays,DaysAttended,AttendedPct,EnrolledPct
3482,2019-2020,167.0,160.0,0.9580838323353293,0.9277777777777778
3,2017-2018,180.0,175.0,0.9722222222222222,1.0
3,2018-2019,180.0,168.0,0.9333333333333333,1.0
3,2019-2020,180.0,178.0,0.9888888888888889,1.0
4,2017-2018,180.0,125.0,0.6944444444444444,1.0
4,2018-2019,103.0,90.0,0.8737864077669902,0.5722222222222222
4529,2021-2022,170.0,138.0,0.8117647058823529,0.9444444444444444
4002,2020-2021,180.0,167.0,0.9277777777777778,1.0
4002,2021-2022,170.0,139.0,0.8176470588235294,0.9444444444444444
5,2017-2018,180.0,165.0,0.9166666666666666,1.0


In [ ]:
# SQL results to dataframe
att_sql = '''
select s.Stu_deID, AcademicYear, sum(ExpectedAttendanceDays) as ExpectedAttendanceDays, sum(DaysAttended) as DaysAttended
, sum(DaysAttended) / sum(ExpectedAttendanceDays) as AttendedPct
, sum(ExpectedAttendanceDays) / 180.0 as EnrolledPct
from att as a 
join ssid as s 
    on a.SSID = s.SSID
group by a.SSID, a.AcademicYear
'''

att_df = pd.read_sql_query(att_sql, cnn)

In [ ]:
# Merge attendance into ELPAC file
elpac = elpac.merge(att_df, how='inner', left_on=('Stu_deID', 'AcademicYear'), right_on=('Stu_deID', 'AcademicYear'))

In [ ]:
# Feature Engineering
elpac['GradeEnrolledPct'] = elpac.GradeLevel + elpac.EnrolledPct.replace(1.0,.999999)

elpac['GradeAttendedPct'] = elpac.GradeLevel + elpac.AttendedPct.replace(1.0,.999999)

### Sort columns for final output

In [ ]:
elpac = elpac.iloc[:,[1, 0, 17, 3, 2, 8, 9, 10, 11, 12, 13, 7, 4, 5, 6, 18, 19, 20, 21, 22, 23, 15, 14, 16]]

### Export ELPAC file to CSV

In [ ]:
elpac.to_csv('/Users/oscargil/Desktop/ADS-599B/Data Folder/elpac.csv', encoding='utf-8', index=False)